In [39]:
import numpy as np
import numpy.typing as npt
import scipy.optimize as opt
import sklearn.datasets as datasets
from dataclasses import dataclass
from typing import Literal, Generator


# Helper Functions


In [40]:

def vcol(vec: npt.NDArray) -> npt.NDArray:
    return vec.reshape(-1, 1)


def vrow(vec: npt.NDArray) -> npt.NDArray:
    return vec.reshape(1, -1)

def split_db_2to1(
    D: npt.NDArray, L: npt.NDArray, seed=0
) -> tuple[tuple[npt.NDArray, npt.NDArray], tuple[npt.NDArray, npt.NDArray]]:
    nTrain = int(D.shape[1] * 2.0 / 3.0)
    np.random.seed(seed)
    idx = np.random.permutation(D.shape[1])
    idxTrain = idx[0:nTrain]
    idxTest = idx[nTrain:]

    DTR = D[:, idxTrain]
    DTE = D[:, idxTest]
    LTR = L[idxTrain]
    LTE = L[idxTest]

    return (DTR, LTR), (DTE, LTE)


def confusion_matrix(y_true: npt.NDArray, y_pred: npt.NDArray) -> npt.NDArray[np.int32]:
    """Compute the confusion matrix for the given true and predicted labels

    Args:
        y_true (npt.NDArray): The true labels
        y_pred (npt.NDArray): The predicted labels

    Returns:
        npt.NDArray: The confusion matrix with the following structure:
            [[TN, FP],
             [FN, TP]]
    """
    return np.array(
        [
            [
                np.sum(np.logical_and(y_true == 0, y_pred == 0)),
                np.sum(np.logical_and(y_true == 0, y_pred == 1)),
            ],
            [
                np.sum(np.logical_and(y_true == 1, y_pred == 0)),
                np.sum(np.logical_and(y_true == 1, y_pred == 1)),
            ],
        ]
    )


def yield_confusion_matrices(
    y_true: npt.NDArray, thresholds: npt.NDArray
) -> Generator[npt.NDArray[np.int32], None, None]:
    indices = np.argsort(thresholds)
    ts = thresholds[indices]
    sorted_y_val = y_true[indices]

    y_pred = np.ones_like(y_true)

    TN = 0
    TP = np.sum(np.logical_and(y_true == 1, y_pred == 1))
    FP = len(y_true) - TP
    FN = 0

    for i in range(1, len(ts)):
        y_pred[i - 1] = 0

        if sorted_y_val[i - 1] == 1:
            TP -= 1
            FN += 1
        else:
            FP -= 1
            TN += 1

        yield np.array([[TN, FP], [FN, TP]])


def optimal_bayes_threshold(pi: float, C_fn: float, C_fp: float) -> float:
    return -np.log((pi * C_fn) / ((1 - pi) * C_fp))


def effective_prior(pi_T: float, C_fn: float, C_fp: float) -> float:
    return (pi_T * C_fn) / (pi_T * C_fn + (1 - pi_T) * C_fp)


def dcf(
    llr: npt.NDArray,
    y_val: npt.NDArray,
    pi: float,
    Cf_n: float,
    Cf_p: float,
    strategy: Literal["optimal"] | Literal["min"] | Literal["manual"],
    normalize=False,
    threshold=0.0,
) -> float:
    """
    Compute the Detection Cost Function (DCF) for the given data and priors.

    Args:
        llr (NDArray): The log-likelihood ratio values.
        y_val (NDArray): The true labels.
        pi (float): The prior probability of a genuine sample.
        Cf_n (float): The cost of false negative.
        Cf_p (float): The cost of false positive.
        strategy (
            Literal["optimal"]
            | Literal["min"]
            | Literal["manual"],
        ): The threshold strategy to use, either "optimal", "min", or "manual".
            Use "optimal" to compute the optimal threshold, "min" to compute the
            minimum DCF value, and "manual" to use the given threshold.
        normalize (bool, optional): Whether to normalize the DCF value.
            Defaults to False.
        threshold (float, optional): The threshold to use if strategy is "manual".
            Does not have any effect if strategy is not "manual". Defaults to 0.0.

    Returns:
        float: The DCF value.
    """

    if strategy == "min":
        # Returns the minimum DCF value calculated over all the possible
        # threhsolds (taken from the log likelihood ratios)
        return min(
            [
                (
                    pi * (cm[1, 0] / cm[1].sum()) * Cf_n
                    + (1 - pi) * (cm[0, 1] / cm[0].sum()) * Cf_p
                )
                / (min(pi * Cf_n, (1 - pi) * Cf_p) if normalize else 1)
                for cm in yield_confusion_matrices(y_val, llr)
            ]
        )
    else:
        threshold = (
            optimal_bayes_threshold(pi, Cf_n, Cf_p)
            if strategy == "optimal"
            else threshold  # if strategy == "manual"
        )

        y_pred = llr > threshold

        cm = confusion_matrix(y_val, y_pred)

        P_fn = cm[1, 0] / cm[1].sum()
        P_fp = cm[0, 1] / cm[0].sum()

        return (pi * P_fn * Cf_n + (1 - pi) * P_fp * Cf_p) / (
            # Normalize the DCF value by dividing it by the best of the two
            # dummy systems: the one that always accepts a test segment and
            # the one that always rejects it.
            min(pi * Cf_n, (1 - pi) * Cf_p)
            if normalize
            else 1  # If normalization is not required, return the raw DCF value
        )


# Numerical Optimization


In [41]:
opt.fmin_l_bfgs_b?

Signature:
opt.fmin_l_bfgs_b(
    func,
    x0,
    fprime=None,
    args=(),
    approx_grad=0,
    bounds=None,
    m=10,
    factr=10000000.0,
    pgtol=1e-05,
    epsilon=1e-08,
    iprint=-1,
    maxfun=15000,
    maxiter=15000,
    disp=None,
    callback=None,
    maxls=20,
)
Docstring:
Minimize a function func using the L-BFGS-B algorithm.

Parameters
----------
func : callable f(x,*args)
    Function to minimize.
x0 : ndarray
    Initial guess.
fprime : callable fprime(x,*args), optional
    The gradient of `func`. If None, then `func` returns the function
    value and the gradient (``f, g = func(x, *args)``), unless
    `approx_grad` is True in which case `func` returns only ``f``.
args : sequence, optional
    Arguments to pass to `func` and `fprime`.
approx_grad : bool, optional
    Whether to approximate the gradient numerically (in which case
    `func` returns only the function value).
bounds : list, optional
    ``(min, max)`` pairs for each element in ``x``, defining


In [42]:
def f(x: npt.ArrayLike) -> float:
    y, z = x # x.shape = (2,)
    return (y + 3)**2 + np.sin(y) + (z + 1)**2

We can call the function with `approx_grad=True` to use numerical gradient approximation.


In [43]:
x, f, d = opt.fmin_l_bfgs_b(f, np.array([0, 0]), approx_grad=True, iprint=1)

In [44]:
print(x, f, d)

[-2.57747138 -0.99999927] -0.3561430123647649 {'grad': array([-1.49324998e-06,  1.46549439e-06]), 'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL', 'funcalls': 21, 'nit': 6, 'warnflag': 0}


We can also compute manually the gradient and return it as a tuple. Doing so leads to less function evaluations and faster convergence.


In [45]:
def f(x: npt.ArrayLike) -> float:
    y, z = x # x.shape = (2,)
    return ((y + 3)**2 + np.sin(y) + (z + 1)**2, np.array([2*(y + 3) + np.cos(y), 2*(z + 1)]))

In [46]:
x, f, d = opt.fmin_l_bfgs_b(f, np.array([0, 0]), approx_grad=False, iprint=1)

In [47]:
print(x, f, d)

[-2.57747137 -0.99999927] -0.3561430123647611 {'grad': array([-1.50318729e-06,  1.46120529e-06]), 'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL', 'funcalls': 7, 'nit': 6, 'warnflag': 0}


## Binary Logistic Regression


### Load the data


In [48]:

def split_db_2to1(D, L, seed=0):
    nTrain = int(D.shape[1] * 2.0 / 3.0)
    np.random.seed(seed)
    idx = np.random.permutation(D.shape[1])
    idxTrain = idx[0:nTrain]
    idxTest = idx[nTrain:]
    DTR = D[:, idxTrain]
    DTE = D[:, idxTest]
    LTR = L[idxTrain]
    LTE = L[idxTest]
    return (DTR, LTR), (DTE, LTE)

In [49]:
def load_iris_binary():
    D, L = datasets.load_iris()['data'].T, datasets.load_iris()['target']
    D = D[:, L != 0] # We remove setosa from D
    L = L[L!=0] # We remove setosa from L
    L[L==2] = 0 # We assign label 0 to virginica (was label 2)
    return D, L

D, L = load_iris_binary()
(DTR, LTR), (DVAL, LVAL) = split_db_2to1(D, L)

## Regularized Logistic Regression objective function


In [131]:
@dataclass
class LogReg:
    DTR : npt.NDArray[np.float64]
    LTR : npt.NDArray[np.int64]
    l : float
    
    def __call__(self, v: npt.ArrayLike) -> tuple[float, npt.NDArray[np.float64]]:
        w, b = v[:-1], v[-1] # v.shape = (DTR.shape[0] + 1,)
        ZTR = 2 * self.LTR - 1
        S = (vcol(w).T @ self.DTR + b).ravel()
        G = -ZTR / (1 + np.exp(ZTR * S))
        
        weights = np.ones_like(LTR) / len(LTR)
        
        
        GW = self.l * w + (weights * vrow(G) * DTR).sum(axis=1)
       
        Gb = np.atleast_1d(np.sum(weights * G))
        
        print(GW, Gb)
 
        # print(np.hstack([GW, np.array(Gb)]))
        # print(np.array(*[GW], Gb))
        return (
            self.l / 2 * np.linalg.norm(w)**2 + np.mean(np.logaddexp(0, -ZTR * S)),
            np.hstack([
                GW, Gb
            ])
            # np.hstack([GW, np.array(Gb)])
        )

In [132]:
logRegObj = LogReg(DTR, LTR, 1.0)

x, f, d = opt.fmin_l_bfgs_b(logRegObj, np.zeros(DTR.shape[0] + 1), approx_grad=False, iprint=1)

[ 0.06212121 -0.00075758  0.23636364  0.1530303 ] [-0.01515152]
[-3.26672503 -1.42378271 -3.03413562 -1.21511724] [-0.5119913]
[-0.25485837 -0.1419905  -0.03631571  0.0511964 ] [-0.06398008]
[-0.18478609 -0.10790619  0.00536226  0.06025429] [-0.0543129]
[0.0724838  0.02860832 0.08454371 0.04006852] [-0.02325641]
[0.03014554 0.01414767 0.01995171 0.00594638] [-0.03136842]
[-0.00063078  0.00162473 -0.01370587 -0.00931415] [-0.03637504]
[-0.04433683 -0.01641391 -0.05929516 -0.02956889] [-0.04314284]
[-0.10160041 -0.03999435 -0.11809211 -0.05585058] [-0.05143672]
[-0.18808489 -0.07533076 -0.20552852 -0.09560219] [-0.06247917]
[-0.28009536 -0.11231009 -0.29554983 -0.13806121] [-0.07051378]
[-0.31063629 -0.12343601 -0.31857985 -0.15195154] [-0.06339899]
[-0.15613043 -0.07404793 -0.20472013 -0.06801235] [-0.02956204]
[-0.06770686 -0.04241141 -0.10074796 -0.01191241] [-0.01248656]
[ 0.00409216 -0.00898229 -0.0204767   0.01535652] [-0.00101694]
[ 0.01017413 -0.00288018 -0.00280636  0.01489581] 

In [119]:
print(x, f, d)

[-0.11040209 -0.0289869  -0.2478711  -0.14950473  2.3109451 ] 0.6316436205354083 {'grad': array([-4.24599635e-06,  1.46396319e-05, -6.41352979e-07,  7.31940209e-06,
        4.56663881e-07]), 'task': 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH', 'funcalls': 22, 'nit': 19, 'warnflag': 0}


### Score


In [120]:
PRIOR = 0.5

In [121]:
w, b = x[:-1], x[-1]

S = vcol(w).T @ DVAL + b

LP = S > 0

#### Error Rate


In [122]:
1 - np.mean(LP == LVAL)

0.1470588235294118

In [56]:
S_llr = S - np.log(PRIOR / (1 - PRIOR))

S_llr = S_llr.T.ravel()

min_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "min", normalize=True)
act_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "optimal", normalize=True)

print(min_dcf, act_dcf)

0.05555555555555555 1.0


## Trying different $\lambda$ values


In [58]:
x, f, d = opt.fmin_l_bfgs_b(LogReg(DTR, LTR, 1e-1), np.zeros(DTR.shape[0] + 1), approx_grad=False, iprint=1)
print(x, f, d)

w, b = x[:-1], x[-1]
S = vcol(w).T @ DVAL + b
LP = S > 0
print(1 - np.mean(LP == LVAL))

[ 0.69395304  0.55971896 -0.96568114 -0.93147582  0.32297832] 0.5608557742663552 {'grad': array([-3.89478999, -3.01272596,  3.8315653 ,  4.36316412, -1.77024887]), 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 101, 'nit': 10, 'warnflag': 2}
0.02941176470588236


C:\Users\occhi\AppData\Local\Temp\ipykernel_1108\285541438.py:11: RuntimeWarning: overflow encountered in exp
  G = -ZTR / (1 + np.exp(ZTR * S))


In [59]:
S_llr = S - np.log(PRIOR / (1 - PRIOR))

S_llr = S_llr.T.ravel()

min_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "min", normalize=True)
act_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "optimal", normalize=True)

print(min_dcf, act_dcf)

0.05555555555555555 1.0


In [60]:
x, f, d = opt.fmin_l_bfgs_b(LogReg(DTR, LTR, 1e-3), np.zeros(DTR.shape[0] + 1), approx_grad=False, iprint=1)
print(x, f, d)

w, b = x[:-1], x[-1]
S = vcol(w).T @ DVAL + b
LP = S > 0
print(1 - np.mean(LP == LVAL))

[ 4.11214704  3.89904297 -5.58198202 -7.24588885  2.95444298] 0.16049877938832988 {'grad': array([ 0.01745257, -0.08442391,  0.00512626,  0.31508666, -0.26770466]), 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 920, 'nit': 50, 'warnflag': 2}
0.08823529411764708


In [61]:
S_llr = S - np.log(PRIOR / (1 - PRIOR))

S_llr = S_llr.T.ravel()

min_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "min", normalize=True)
act_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "optimal", normalize=True)

print(min_dcf, act_dcf)

0.1111111111111111 0.2222222222222222


## Prior-weighted logistic regression and calibration


In [ ]:
@dataclass
class LogRegWeighted:
    DTR : npt.NDArray[np.float64]
    LTR : npt.NDArray[np.int64]
    l : float
    pi : float
    
    def __call__(self, v: npt.ArrayLike) -> tuple[float, npt.NDArray[np.float64]]:
        w, b = v[:-1], v[-1]
        
        ZTR = 2 * self.LTR - 1
        S = (vcol(w).T @ self.DTR + b).ravel()
        
        n_T = np.sum(self.LTR == 1)
        n_F = np.sum(self.LTR == 0)
        
        weights = np.where(self.LTR == 1, self.pi / n_T, (1 - self.pi) / n_F)

        return self.l / 2 * np.linalg.norm(w)**2 + np.sum(weights * np.logaddexp(0, -ZTR * S))
        

In [ ]:
x, f, d = opt.fmin_l_bfgs_b(LogRegWeighted(DTR, LTR, 1.0, 0.5), np.zeros(DTR.shape[0] + 1), approx_grad=True, iprint=1)

print(x, f, d)

PRIOR = 0.8

w, b = x[:-1], x[-1]

S = vcol(w).T @ DVAL + b

LP = S > 0

print(1 - np.mean(LP == LVAL))

S_llr = S - np.log(PRIOR / (1 - PRIOR))

S_llr = S_llr.T.ravel()

min_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "min", normalize=True)
act_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "optimal", normalize=True)

print(min_dcf, act_dcf)

[-0.11036541 -0.0290465  -0.24798317 -0.14963204  2.25695248] 0.632017629678834 {'grad': array([-9.35918010e-06, -3.84137166e-06, -5.02931030e-06, -9.65894032e-07,
       -2.25375275e-06]), 'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL', 'funcalls': 138, 'nit': 19, 'warnflag': 0}
0.08823529411764708
0.1111111111111111 0.16666666666666666


In [ ]:
x, f, d = opt.fmin_l_bfgs_b(LogRegWeighted(DTR, LTR, 1e-1, 0.8), np.zeros(DTR.shape[0] + 1), approx_grad=True, iprint=1)
print(x, f, d)

PRIOR = 0.8

w, b = x[:-1], x[-1]

S = vcol(w).T @ DVAL + b

LP = S > 0

print(1 - np.mean(LP == LVAL))

S_llr = S - np.log(PRIOR / (1 - PRIOR))

S_llr = S_llr.T.ravel()

min_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "min", normalize=True)
act_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "optimal", normalize=True)

print(min_dcf, act_dcf)

[-0.25343795 -0.04809752 -0.94125105 -0.64605667  8.58673869] 0.36062613082254713 {'grad': array([1.84852134e-06, 6.16173779e-07, 1.52100554e-06, 6.82787157e-07,
       1.99840128e-07]), 'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL', 'funcalls': 174, 'nit': 25, 'warnflag': 0}
0.38235294117647056
0.05555555555555555 0.7222222222222222


In [ ]:
x, f, d = opt.fmin_l_bfgs_b(LogRegWeighted(DTR, LTR, 1e-3, 0.8), np.zeros(DTR.shape[0] + 1), approx_grad=True, iprint=1)
print(x, f, d)

PRIOR = 0.8

w, b = x[:-1], x[-1]

S = vcol(w).T @ DVAL + b

LP = S > 0

print(1 - np.mean(LP == LVAL))

S_llr = S - np.log(PRIOR / (1 - PRIOR))

S_llr = S_llr.T.ravel()

min_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "min", normalize=True)
act_dcf = dcf(S_llr, LVAL, 0.8, 1, 1, "optimal", normalize=True)

print(min_dcf, act_dcf)

[ 1.51852491  0.65126024 -4.3227396  -6.46160908 21.91047562] 0.09401035003010494 {'grad': array([-4.92245137e-06, -2.37310170e-06, -4.55330221e-06, -1.32810430e-06,
       -8.36830536e-07]), 'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL', 'funcalls': 324, 'nit': 46, 'warnflag': 0}
0.11764705882352944
0.16666666666666666 0.2222222222222222
